In [27]:
# !pip install pydantic
# !pip install langchain langchain_core langchain_openai 
# !pip install python-dotenv

In [28]:
from pydantic import BaseModel, Field, validator
from typing import List

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
class TaskResponse(BaseModel):
    score: float  # We set the type as float but will validate further
    concerns: List[str]

    @validator('score')
    def ensure_score_is_numeric(cls, v):
        if not isinstance(v, (int, float)):
            raise ValueError('Score must be numeric (integer or float).')
        if v < 0 or v > 10:
            raise ValueError('Score must be between 0 and 10.')
        return v

class CohesionAndCoherence(BaseModel):
    score: float
    concerns: List[str]

    @validator('score')
    def ensure_score_is_numeric(cls, v):
        if not isinstance(v, (int, float)):
            raise ValueError('Score must be numeric (integer or float).')
        if v < 0 or v > 10:
            raise ValueError('Score must be between 0 and 10.')
        return v

class Vocabulary(BaseModel):
    score: float
    concerns: List[str]

    @validator('score')
    def ensure_score_is_numeric(cls, v):
        if not isinstance(v, (int, float)):
            raise ValueError('Score must be numeric (integer or float).')
        if v < 0 or v > 10:
            raise ValueError('Score must be between 0 and 10.')
        return v

class Grammar(BaseModel):
    score: float
    concerns: List[str]

    @validator('score')
    def ensure_score_is_numeric(cls, v):
        if not isinstance(v, (int, float)):
            raise ValueError('Score must be numeric (integer or float).')
        if v < 0 or v > 10:
            raise ValueError('Score must be between 0 and 10.')
        return v

class Overall(BaseModel):
    score: float
    main_issue: str

    @validator('score')
    def ensure_score_is_numeric(cls, v):
        if not isinstance(v, (int, float)):
            raise ValueError('Score must be numeric (integer or float).')
        if v < 0 or v > 10:
            raise ValueError('Score must be between 0 and 10.')
        return v

class WritingAssessment(BaseModel):
    task_response: TaskResponse
    cohesion_and_coherence: CohesionAndCoherence
    vocabulary: Vocabulary
    grammar: Grammar
    overall: Overall
    reference_answers: List[str]



/tmp/ipykernel_23443/2872923642.py:5: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('score')
/tmp/ipykernel_23443/2872923642.py:17: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('score')
/tmp/ipykernel_23443/2872923642.py:29: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for mo

In [31]:
# Example usage with valid scores
example_assessment = WritingAssessment(
    task_response=TaskResponse(score=8, concerns=["Clarity", "Detail"]),
    cohesion_and_coherence=CohesionAndCoherence(score=7.5, concerns=["Transition", "Flow"]),
    vocabulary=Vocabulary(score=8.2, concerns=["Limited range of words"]),
    grammar=Grammar(score=7, concerns=["Sentence structure"]),
    overall=Overall(score=7.8, main_issue="Repetitive wording"),
    reference_answers=["Example reference answer 1", "Example reference answer 2"]
)

print(example_assessment)

task_response=TaskResponse(score=8.0, concerns=['Clarity', 'Detail']) cohesion_and_coherence=CohesionAndCoherence(score=7.5, concerns=['Transition', 'Flow']) vocabulary=Vocabulary(score=8.2, concerns=['Limited range of words']) grammar=Grammar(score=7.0, concerns=['Sentence structure']) overall=Overall(score=7.8, main_issue='Repetitive wording') reference_answers=['Example reference answer 1', 'Example reference answer 2']


In [32]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [33]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [34]:
from prompts import PARAPHRASE_GRADE_PROMPT_TEMPLATE, WRITING_ASSESSMENT_FORMAT

In [35]:
paraphrase_grade_prompt = PromptTemplate(
    template=PARAPHRASE_GRADE_PROMPT_TEMPLATE,
    # input_variables=["original_text"]
    partial_variables={"WRITING_ASSESSMENT_FORMAT": WRITING_ASSESSMENT_FORMAT}
)

In [36]:
paraphrase_grade_prompt

PromptTemplate(input_variables=['original_text', 'user_paraphrase'], input_types={}, partial_variables={'WRITING_ASSESSMENT_FORMAT': '\n{\n  "task_response": {\n    "score": "PLACEHOLDER_SCORE",\n    "concerns": [\n      "PLACEHOLDER_CONCERN_1",\n      "PLACEHOLDER_CONCERN_2"\n    ]\n  },\n  "cohesion_and_coherence": {\n    "score": "PLACEHOLDER_SCORE",\n    "concerns": [\n      "PLACEHOLDER_CONCERN_3",\n      "PLACEHOLDER_CONCERN_4"\n    ]\n  },\n  "vocabulary": {\n    "score": "PLACEHOLDER_SCORE",\n    "concerns": [\n      "PLACEHOLDER_VOCAB_CONCERN"\n    ]\n  },\n  "grammar": {\n    "score": "PLACEHOLDER_SCORE",\n    "concerns": [\n      "PLACEHOLDER_GRAMMAR_CONCERN"\n    ]\n  },\n  "overall": {\n    "score": "PLACEHOLDER_SCORE",\n    "main_issue": "PLACEHOLDER_MAIN_ISSUE"\n  },\n  "reference_answers": [\n    "PLACEHOLDER_REFERENCE_ANSWERS\n  ]\n}\n\n'}, template="\nYou are an expert in scoring IELTS test, exam. \nGiven the original text, and user's paraphrase for it, your task is t

In [37]:
parser = PydanticOutputParser(pydantic_object=WritingAssessment)

In [38]:
chain = paraphrase_grade_prompt | llm | parser
# chain = paraphrase_grade_prompt | llm 

In [39]:
input = {
    "original_text": "Tesla has unveiled the Cybercab, a new electric vehicle dedicated to self-driving that lacks a steering wheel or pedals.",
    "user_paraphrase": "Tesla has introduced the Cybercab, a self-driving electric vehicle that does not have a steering wheel or pedals."
    }

In [40]:
print(paraphrase_grade_prompt.invoke(input).text)


You are an expert in scoring IELTS test, exam. 
Given the original text, and user's paraphrase for it, your task is to score quality of user's paraphrase, correct it if needed and provide reference pharase. 
Note: 
- Your output text should be preserve the format of original text
- Your output should be as the follows:
- The score in range 0-9 like IELTS grading
- Provide at least 3 answer for reference answer, apart from original text

{
  "task_response": {
    "score": "PLACEHOLDER_SCORE",
    "concerns": [
      "PLACEHOLDER_CONCERN_1",
      "PLACEHOLDER_CONCERN_2"
    ]
  },
  "cohesion_and_coherence": {
    "score": "PLACEHOLDER_SCORE",
    "concerns": [
      "PLACEHOLDER_CONCERN_3",
      "PLACEHOLDER_CONCERN_4"
    ]
  },
  "vocabulary": {
    "score": "PLACEHOLDER_SCORE",
    "concerns": [
      "PLACEHOLDER_VOCAB_CONCERN"
    ]
  },
  "grammar": {
    "score": "PLACEHOLDER_SCORE",
    "concerns": [
      "PLACEHOLDER_GRAMMAR_CONCERN"
    ]
  },
  "overall": {
    "score": 

In [41]:
response = chain.invoke(input)
response

WritingAssessment(task_response=TaskResponse(score=7.0, concerns=['Minor changes in word choice', 'Overall captures the main idea']), cohesion_and_coherence=CohesionAndCoherence(score=8.0, concerns=['Clear and logical structure', 'Maintains original meaning']), vocabulary=Vocabulary(score=8.0, concerns=['Good use of vocabulary', 'Appropriate synonyms']), grammar=Grammar(score=8.0, concerns=['Correct verb tense and agreement', 'Proper sentence structure']), overall=Overall(score=8.0, main_issue='Minor word choice changes, but overall well-paraphrased'), reference_answers=['Tesla has unveiled the Cybercab, an electric self-driving vehicle that comes without a steering wheel or pedals.', 'Tesla has revealed the Cybercab, a new self-driving electric car that lacks a steering wheel or pedals.', 'Tesla has introduced the Cybercab, an electric self-driving vehicle that is devoid of a steering wheel or pedals.'])

In [42]:
import json
print(json.dumps(response.model_dump(), indent=2))

{
  "task_response": {
    "score": 7.0,
    "concerns": [
      "Minor changes in word choice",
      "Overall captures the main idea"
    ]
  },
  "cohesion_and_coherence": {
    "score": 8.0,
    "concerns": [
      "Clear and logical structure",
      "Maintains original meaning"
    ]
  },
  "vocabulary": {
    "score": 8.0,
    "concerns": [
      "Good use of vocabulary",
      "Appropriate synonyms"
    ]
  },
  "grammar": {
    "score": 8.0,
    "concerns": [
      "Correct verb tense and agreement",
      "Proper sentence structure"
    ]
  },
  "overall": {
    "score": 8.0,
    "main_issue": "Minor word choice changes, but overall well-paraphrased"
  },
  "reference_answers": [
    "Tesla has unveiled the Cybercab, an electric self-driving vehicle that comes without a steering wheel or pedals.",
    "Tesla has revealed the Cybercab, a new self-driving electric car that lacks a steering wheel or pedals.",
    "Tesla has introduced the Cybercab, an electric self-driving vehic